In [1]:
packages = [
    "torch==2.0.1",
    "torch-geometric==2.3.1",
    "numpy==1.23.0",
    "pandas==1.5.3"
]
import warnings
warnings.filterwarnings("ignore")
# Instalación de cada paquete uno por uno
for package in packages:
    !pip install {package}

  Using cached numpy-1.23.0.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build numpy


  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [213 lines of output]
      setup.py:71: RuntimeWarning: NumPy 1.23.0 may not yet support Python 3.11.
        warnings.warn(
      Running from numpy source directory.
      setup.py:86: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
        import numpy.distutils.command.sdist
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\bit_generator.pyx
      Processing numpy/random\mtrand.pyx
      Proces

  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3


In [2]:
!pip install torch
!pip install --upgrade numpy pandas
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch.distributions import Categorical
import numpy as np
import pandas as pd

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


In [3]:
#set input dates
date = "_2024-11-04"
truck = "VAN"

In [66]:
# Specify the file name
file_name1 = "rate_q2_west_"+truck+date+".csv"
file_name2 = "duration_west.csv"
file_name3 = "prob_west_"+truck+date+".csv"
file_name4 = "load_av_west_"+truck+date+".csv"
file_name5 = "distance_west.csv"
file_name6 = "diesel_west"+date+".csv"
file_name7 = "labels_west.csv"
# Read the Excel file into a DataFrame
rate_matrix = pd.read_csv(file_name1,header= None)
time_matrix = pd.read_csv(file_name2,header = None)
markov_matrix = pd.read_csv(file_name3,header=None)
loads_matrix = pd.read_csv(file_name4,header=None)
distance_matrix = pd.read_csv(file_name5,header=None)
diesel_matrix = pd.read_csv(file_name6,header=None)
hub_labels = pd.read_csv(file_name7,header=None)
initial_node = 9
time_limit = 60

In [80]:
revenue_matrix = rate_matrix * distance_matrix
revenue_matrix[loads_matrix <= 1] = 0
diesel_matrix = diesel_matrix / 6
cost_matrix = distance_matrix * diesel_matrix
cost_matrix = cost_matrix + 163
reward_matrix = revenue_matrix - cost_matrix
time_matrix *= 0.9

In [68]:
class GraphEnvironment:
    def __init__(self, reward_matrix, time_matrix, markov_matrix, initial_node, time_limit, loads_matrix):
        self.reward_matrix = reward_matrix
        self.time_matrix = time_matrix
        self.markov_matrix = markov_matrix
        self.initial_node = initial_node
        self.time_limit = time_limit
        self.loads_matrix = loads_matrix
        self.num_nodes = reward_matrix.shape[0]
        self.edge_index = self.generate_edge_index()
        
    def generate_edge_index(self):
        edges = []
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if i != j:
                    edges.append((i, j))
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_index = edge_index[1]
        return edge_index

    def reset(self):
        self.current_node = self.initial_node
        self.remaining_time = self.time_limit
        self.visited_nodes = [self.initial_node]
        self.visited_edges = []
        self.total_reward = 0
        return self.get_state()

    def get_state(self):
        return torch.tensor([self.current_node, self.remaining_time], dtype=torch.float32)

    def is_terminal(self):
        # Helper function for terminal conditions
        return (self.remaining_time <= -self.time_limit * 0.5) and (self.current_node == self.initial_node)

    def step(self, action):
        next_node = action.item()
        edge = (self.current_node, next_node)

        if edge in self.visited_edges or self.current_node == next_node:
            # Handle invalid moves (e.g., repeat or self-loop)
            return self.get_state(), 0, False, {}, self.visited_nodes

        self.visited_edges.append(edge)
        reward = self.reward_matrix.iloc[self.current_node, next_node]
        time_consumption = self.time_matrix.iloc[self.current_node, next_node]
        self.total_reward += reward
        self.remaining_time -= time_consumption
        self.current_node = next_node
        self.visited_nodes.append(next_node)

        if self.is_terminal():
            done = True
            # Optionally include final rewards or reset if necessary
            return self.get_state(), reward, done, {}, self.visited_nodes
            
        # Check if time to return to start
        if -self.time_limit * 0.25 < self.remaining_time <= 0:
            edge = (self.current_node, self.initial_node)
            reward = self.reward_matrix.iloc[self.current_node, self.initial_node]
            #loads = self.loads_matrix.iloc[self.current_node, self.initial_node]
            time_consumption = self.time_matrix.iloc[self.current_node, self.initial_node]
            self.total_reward += reward
            self.remaining_time -= time_consumption
            self.current_node = self.initial_node
            self.visited_nodes.append(self.initial_node)
            done = (self.current_node == self.initial_node)
            return self.get_state(), reward, done, {},self.visited_nodes
        elif (len(self.visited_nodes) == self.num_nodes) or (self.remaining_time <= -self.time_limit*0.25):
            self.current_node = self.initial_node
            self.remaining_time = self.time_limit
            self.visited_nodes = [self.initial_node]
            self.visited_edges = []
            self.total_reward = 0
            return self.get_state(), 0, False, {},self.visited_nodes

        # Not terminal and valid move
        return self.get_state(), reward, False, {}, self.visited_nodes

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, x):
        # Compute attention scores and normalize
        scores = self.attention_weights(x)
        weights = F.softmax(scores, dim=1)
        # Apply attention weights to the input
        attended_output = torch.bmm(weights.transpose(1, 2), x).squeeze(1)
        return attended_output

class AttentionDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttentionDQN, self).__init__()
        
        # Initial linear layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
        # Attention layers
        self.attention_layer = nn.Linear(hidden_size, hidden_size)
        
        # Final linear layers after attention
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, state):
        # Standard forward pass through initial layers
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        
        # Attention mechanism
        # Expand dimensions to create compatibility for bmm
        x_expanded = x.unsqueeze(1)  # Shape: [batch_size, 1, hidden_size]
        
        attention_scores = F.softmax(self.attention_layer(x), dim=-1)  # Shape: [batch_size, hidden_size]
        attention_scores = attention_scores.unsqueeze(2)  # Shape: [batch_size, hidden_size, 1]
        
        # Apply attention to the input
        attended_output = torch.bmm(x_expanded, attention_scores).squeeze(2)  # Shape: [batch_size, hidden_size]
        
        # Continue forward pass after attention
        x = F.relu(self.fc3(attended_output))
        x = self.fc4(x)
        return x


# class SimpleDQNWithAttention(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(SimpleDQNWithAttention, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
#         self.fc3 = nn.Linear(hidden_size, hidden_size)
#         self.fc4 = nn.Linear(hidden_size, hidden_size)
#         self.attention = Attention(hidden_size)
#         self.output = nn.Linear(hidden_size, output_size)

#     def forward(self, state):
#         x = F.relu(self.fc1(state))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#         x = F.relu(self.fc4(x))
#        x = x.unsqueeze(0)  # Adding batch dimension
#        x = self.attention(x)  # Apply attention
#       x = self.output(x)
#        return x

class SimpleDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDQN, self).__init__()
        
        # Input layer: maps input of size 2 to the hidden size
        self.fc1 = nn.Linear(input_size, hidden_size)
        
        # Attention layer: self-attention based mechanism to align with hidden size
        self.attention_layer = nn.Linear(hidden_size, hidden_size)  # Matches the hidden dimension
        self.attention_score = nn.Linear(hidden_size, 1)
        
        # Additional fully connected layers
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)  # Maps to the output size for Q-values
        
    def forward(self, x):
        # First layer transformation
        x = F.relu(self.fc1(x))  # Project input to hidden size
        
        # Attention mechanism
        attention_scores = F.softmax(self.attention_score(x), dim=1)
        attended_output = attention_scores * x  # Element-wise attention application
        
        # Process through the remaining layers
        x = F.relu(self.fc2(attended_output))
        return self.fc3(x)


In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)
# Define hyperparameters
learning_rate = 0.001
gamma = 0.5
epsilon_start = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
num_episodes = 500
hidden_size = env.num_nodes * (env.num_nodes - 1)  # Set according to number of arcs

# Initialize the model with attention mechanism
input_size = 2  # state dimensions: current node, remaining time
output_size = len(env.edge_index)  # number of possible actions (edges)
model = SimpleDQN(input_size, hidden_size, output_size)  # Using the new Attention DQN
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Set epsilon
epsilon = epsilon_start

# Action selection with epsilon-greedy policy
def select_action(state, epsilon):
    if np.random.rand() < epsilon:
        a_type = 1
        options = env.markov_matrix.iloc[int(state[0].item())]
        probabilities = options / options.sum()
        action = torch.tensor(int(np.random.choice(options.index, p=probabilities.values) + ((state[0].item()) * (env.num_nodes - 1))))
        return action, a_type
    else:
        with torch.no_grad():
            q_values = model(state.unsqueeze(0))
            a_type = 2
            return q_values.argmax(dim=-1).unsqueeze(0), a_type


In [71]:
# Training loop
for i in [9]:  # Iterate through each possible starting node
    stop_counter = 0
    best_reward = 0
    last_reward = 0
    epsilon = epsilon_start
    episode = 0
    patience_counter = 100

    while stop_counter < 10 and episode < num_episodes and epsilon > epsilon_min and patience_counter > 0:
        initial_node = i
        time_limit = 45  # Define time limit for each episode
        env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit, loads_matrix)
        state = env.reset()
        done = False
        total_reward = 0

        # Adjust epsilon for each episode
        epsilon = max(epsilon * epsilon_decay, epsilon_min)

        while not done:
            # Choose an action using epsilon-greedy policy
            action, _ = select_action(state, epsilon)
            new_node = env.edge_index[action.item()]

            # Execute the action in the environment
            next_state, reward, done, _, visited_nodes = env.step(new_node)
            total_reward += reward

            # Calculate the target
            with torch.no_grad():
                # Reshape `next_state` to fit the model's expected input (batch size, feature size)
                next_state_expanded = next_state.unsqueeze(0)  # Adding batch dimension if needed

                # Compute target
                if done:
                    target = torch.tensor(reward, dtype=torch.float32)
                else:
                    next_q_values = model(next_state_expanded)  # Predict Q-values for the next state
                    max_next_q_value = next_q_values.max()  # Get the max Q-value for the best action in the next state
                    target = torch.tensor(reward, dtype=torch.float32) + gamma * max_next_q_value


            # Update model based on state-action pair and loss
            state_expanded = state.unsqueeze(0)
            q_values = model(state_expanded).squeeze(0)
            loss = loss_fn(q_values[action.item()].float(), target.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update state for the next step
            state = next_state

        # Check for reward improvement and update stopping criteria
        if total_reward > best_reward:
            patience_counter = 100 if total_reward > best_reward * 1.1 else patience_counter - 1
            best_reward = total_reward
        else:
            patience_counter -= 1

        stop_counter = stop_counter + 1 if last_reward == total_reward else 0
        last_reward = total_reward

        # Print progress
        print(f"Episode {episode + 1}, Total Reward: {total_reward}, Start Node: {i}, Visited Nodes: {visited_nodes}")
        episode += 1

Episode 1, Total Reward: -439.94999999999993, Start Node: 9, Visited Nodes: [9, 12, 9, 13, 5, 9]
Episode 2, Total Reward: 1761.6916666666666, Start Node: 9, Visited Nodes: [9, 12, 10, 13, 34, 9]
Episode 3, Total Reward: 1380.6833333333336, Start Node: 9, Visited Nodes: [9, 12, 9, 11, 9, 51, 9]
Episode 4, Total Reward: 1146.4599999999998, Start Node: 9, Visited Nodes: [9, 12, 11, 5, 12, 9, 9]
Episode 5, Total Reward: 2031.1166666666668, Start Node: 9, Visited Nodes: [9, 12, 10, 8, 12, 11, 9, 11, 5, 1, 34, 9]
Episode 6, Total Reward: 784.3616666666668, Start Node: 9, Visited Nodes: [9, 12, 9, 7, 10, 13, 9]
Episode 7, Total Reward: 3247.0150000000003, Start Node: 9, Visited Nodes: [9, 12, 11, 13, 15, 9]
Episode 8, Total Reward: 788.2000000000002, Start Node: 9, Visited Nodes: [9, 12, 11, 53, 9]
Episode 9, Total Reward: 1149.2, Start Node: 9, Visited Nodes: [9, 5, 10, 9, 7, 10, 0, 9]
Episode 10, Total Reward: 297.38, Start Node: 9, Visited Nodes: [9, 12, 10, 13, 42, 9]
Episode 11, Total Re

Display Results and Route Solutions

In [51]:
def calculate_days(route, time_matrix):
  total_time = 0
  for i in range(0,len(route)-1):
    lane_time = time_matrix.iloc[route[i],route[i+1]]
    total_time = total_time + lane_time + 2
  days = int(np.floor(total_time / 11))
  return days

In [52]:
def visualize_route (route,hub_labels):
  visualized_route = []
  for i in range(0,len(route)):
    name = hub_labels.at[route[i], 0]
    visualized_route.append(name)
  return visualized_route

In [53]:
def save_best(route, reward, best_profits, most_profit_routes):
    if reward > best_profits.min():
        # Find the index of the lowest value in best_profits
        min_index = best_profits.idxmin()
        
        # Replace the lowest value with the new reward
        best_profits[min_index] = reward
        
        # Replace the corresponding route in most_profit_routes
        most_profit_routes[min_index] = route
        
    return best_profits, most_profit_routes

In [79]:
type1=0
type2=0
type3=0
type4=0
travel_times = [20,40,60]
solutions = len(travel_times)
most_profit_routes = [0,0,0,0,0]
best_profits = [1,2,3,4,5]
# Inference loop
for i in [9]:
    for j in range(0,solutions):
      taken_actions = []
      # Reset environment to get initial state
      # Initialize environment
      initial_node=i
      time_limit=travel_times[j]
      env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)
      state = env.reset()
      # Episode loop
      done = False
      q_values = model(state.unsqueeze(0))
      while not done:
          # Select action using the trained model
          with torch.no_grad():
              q_values = model(state.unsqueeze(0))
              # Apply action mask to zero out probabilities of previously taken actions
              action_mask = torch.zeros_like(q_values)
              # Set probabilities of previously taken actions to -infinity
              action_mask[0,taken_actions] = float('-1000000')
              masked_q_values = q_values + action_mask
              probabilities = F.softmax(masked_q_values, dim=-1)
              action = torch.multinomial(probabilities, num_samples=1).unsqueeze(0)
          # Take action in the environment
          new_node = env.edge_index[action[0].item()]
          next_state, reward, done, _,route = env.step(new_node)
          # Update current state
          state = next_state
          # Update list of previously taken actions
          taken_actions.append(action[0].item())
          if done :
            total_reward =0
            total_revenue = 0
            total_time = 0
            feasible = 1
            steps = len(route)-1
            for j in range(0,steps):
              reward = reward_matrix.iloc[route[j],route[j+1]]
              revenue = revenue_matrix.iloc[route[j],route[j+1]]
              time = time_matrix.iloc[route[j],route[j+1]]
              total_reward = total_reward + reward
              total_revenue = total_revenue + revenue
              total_time = total_time + time
              loads = loads_matrix.iloc[route[j],route[j+1]]
              if loads < 2:
                feasible = 0
      if total_reward > 0 and feasible == 1:
        type1=type1+1
        #best_profits, most_profit_routes = save_best(route, total_reward, best_profits, most_profit_routes)
        
      elif total_reward > 0 and feasible == 0:
        type2=type2+1
      elif total_reward <= 0 and feasible == 1:
        type3=type3+1
      else:
        type4=type4+1
      days = calculate_days(route, time_matrix)
      visualized_route = visualize_route(route,hub_labels)
      # Output episode solution
      print("Route:",visualized_route,"Profit:",total_reward,"Time:",total_time,"Feasible:",feasible)

Route: ['Los Angeles (CA)', 'Moses Lake (WA)', 'Los Angeles (CA)'] Profit: 3908.5444444444443 Time: 46.5020375 Feasible: 1
Route: ['Los Angeles (CA)', 'Flagstaff (AZ)', 'Phoenix (AZ)', 'Stockton (CA)', 'Carson (WA, OR)', 'Los Angeles (CA)'] Profit: 4169.343888888889 Time: 59.901355 Feasible: 1
Route: ['Los Angeles (CA)', 'Moses Lake (WA)', 'Hazleton (PA, NJ)', 'Los Angeles (CA)'] Profit: 9947.511111111111 Time: 129.4679475 Feasible: 1
